# 内容说明
来自刘建平博客中的tensorflow版本，拿过来运行一下，博客园：https://www.cnblogs.com/pinard/p/9163481.html  
但是自己不能理解最后评分的预测值，基本都是零点零几这样，肯定是没法用RMSE误差来衡量的。因为完全没用到实际评分rui，只用到了一个电影是否已经评分的状态

In [2]:
import pandas as pd
import numpy as np
import os
import random
from collections import defaultdict

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

## 1. 读取评分数据
只用读取.inter评分文件即可  
建立评分字典user_ratings，每个元素的“键”为user_id，“值”为该用户评分过的item_id集合  
遍历.inter文件的每一行数据，记录对应关系，同时更新uer_id和item_id的最大值，便于获得user和item的数目

In [3]:
# 定义函数，加载数据
def load_data(data_path):
    
    # Python中通过Key访问字典，当Key不存在时，会引发‘KeyError'异常。
    # 为了避免这种情况的发生，可以使用collections类中的defaultdict()方法来为字典提供默认值
    user_ratings = defaultdict(set)
    
    # 读取文件
    inter = pd.read_csv(data_path, delimiter='\t', engine='python')
    df = pd.DataFrame(inter)
    max_u_id = -1
    max_i_id = -1
    
    # 遍历每一行评分数据
    for index, row in df.iterrows():
        u = row['user_id:token']
        i = row['item_id:token']
        user_ratings[u].add(i)
        max_u_id = max(u, max_u_id)
        max_i_id = max(i, max_i_id)
        
    # 返回user数目，item数目，评分字典
    return max_u_id, max_i_id, user_ratings 


# 调用函数
data_path = '../dataset/ml-100k/ml-100k.inter'
n_user, n_item, user_ratings = load_data(data_path)
print(n_user, n_item)
print(user_ratings)

943 1682
defaultdict(<class 'set'>, {196: {257, 8, 393, 13, 269, 655, 1022, 663, 25, 153, 411, 285, 286, 287, 428, 173, 306, 692, 66, 67, 580, 70, 202, 845, 340, 1241, 94, 1118, 108, 238, 111, 1007, 110, 242, 116, 762, 251, 381, 382}, 186: {1033, 12, 1042, 1046, 540, 31, 546, 550, 38, 554, 44, 53, 566, 55, 568, 56, 1083, 71, 588, 77, 591, 79, 595, 596, 95, 98, 100, 106, 117, 118, 121, 147, 148, 159, 684, 689, 177, 1213, 203, 717, 225, 226, 1253, 742, 237, 754, 243, 250, 1277, 257, 770, 258, 263, 269, 281, 288, 291, 294, 295, 298, 299, 300, 302, 303, 306, 820, 1336, 829, 322, 327, 330, 331, 332, 333, 338, 356, 1385, 880, 887, 1399, 385, 405, 406, 925, 934, 939, 977, 470, 983, 988, 477, 1016}, 22: {2, 515, 4, 523, 526, 17, 21, 24, 29, 546, 550, 554, 50, 53, 566, 568, 62, 68, 79, 80, 85, 89, 94, 96, 105, 109, 110, 117, 118, 121, 636, 127, 128, 648, 651, 144, 153, 154, 665, 161, 163, 167, 168, 683, 172, 173, 684, 687, 176, 175, 688, 174, 692, 181, 184, 186, 187, 194, 195, 712, 201, 202, 20

## 2. 选择用于测试的一个已评分item
下面我们会对每一个用户u，在user_ratings中随机找到他评分过的一部电影i，保存在user_ratings_test，后面构造训练集和测试集需要用到。  
1. 构造训练集时，不能选择user_ratings_test中的 i ，因为已用于测试集了
2. 构造测试集时，user_ratings_test给每个 u 选出了一个已评分 i ,找到所有未评分的 j 来组成三元组

In [4]:
# 定义函数，生成测试数据
def generate_test(user_ratings):
    
    # 初始化字典
    user_test = dict()
    
    # 遍历评分字典的键和值
    for u, i_set in user_ratings.items():
        # user_tset的键为u，值为随机选取的一个已评分item_id
        # random.sample(user_ratings[u], 1)[0]，user_ratings[u]为用户u已评分的item_id集合，1代表选1个
        # [0]是因为random的结果为列表，[0]取出了列表中仅有的一个元素
        user_test[u] = random.sample(user_ratings[u], 1)[0]
    
    # 返回测试数据字典
    return user_test


# 调用函数
user_ratings_test = generate_test(user_ratings)
print(user_ratings_test)

{196: 1118, 186: 257, 22: 430, 244: 324, 166: 984, 298: 527, 115: 922, 253: 188, 305: 131, 6: 204, 62: 652, 286: 1053, 200: 580, 210: 28, 224: 582, 303: 98, 122: 86, 194: 425, 291: 774, 234: 519, 119: 209, 167: 1306, 299: 294, 308: 928, 95: 215, 38: 153, 102: 38, 63: 25, 160: 1223, 50: 286, 301: 121, 225: 482, 290: 243, 97: 429, 157: 268, 181: 1383, 278: 269, 276: 803, 7: 479, 10: 205, 284: 286, 201: 1137, 287: 845, 246: 155, 242: 283, 249: 24, 99: 100, 178: 1258, 251: 595, 81: 544, 260: 270, 25: 173, 59: 77, 72: 177, 87: 866, 42: 468, 292: 174, 20: 252, 13: 506, 138: 182, 60: 180, 57: 245, 223: 289, 189: 185, 243: 655, 92: 1023, 241: 302, 254: 415, 293: 282, 127: 286, 222: 7, 267: 810, 11: 15, 8: 243, 162: 358, 279: 1231, 145: 684, 28: 196, 135: 475, 32: 50, 90: 1199, 216: 412, 250: 338, 271: 124, 265: 748, 198: 154, 168: 7, 110: 338, 58: 709, 237: 58, 94: 56, 128: 275, 44: 252, 264: 56, 41: 156, 82: 1134, 262: 393, 174: 347, 43: 393, 84: 597, 269: 187, 259: 97, 85: 69, 213: 985, 121:

## 3. 生成批训练集三元组
batch批的大小默认值为512  
每条训练数据是[u, i, j]的三元组形式，注意 i 不能选择user_ratings_test中用于测试的那个

In [5]:
# 定义函数，生成批训练集
# 参数：评分字典user_ratings，测试集user_ratings_test(不能出现在训练集)，item数目n_item，批大小batch_szie
def generate_train_batch(user_ratings, user_ratings_test, n_item, batch_size=512):
    
    # 初始化训练集
    train = []
    
    # 生词batch_size大小的训练集
    for b in range(batch_size):
        
        # 随机选择u，即user_id
        u = random.sample(user_ratings.keys(), 1)[0]
        
        # 随机选择i，即该user已评分过的一个item_id
        i = random.sample(user_ratings[u], 1)[0]  # 在该user评分过的item中随机选择一个
        while i == user_ratings_test[u]:  # 防止正好选中了用于测试集的
            i = random.sample(user_ratings[u], 1)[0]
        
        # 随机选择j，即该user未评分过的一个item_id
        j = random.randint(1, n_item)  # 随机生成item_id
        while j in user_ratings[u]:  # 防止选到了已评分的
            j = random.randint(1, n_item)
        
        # 以列表方式存储三元组
        train.append([u, i, j])
    
    # 以矩阵方式返回批训练集
    return np.asarray(train)


# 调用函数
train_batch = generate_train_batch(user_ratings, user_ratings_test, n_item, batch_size=512)
print(train_batch)

[[ 905  475   39]
 [ 116 1220  197]
 [ 151  523  297]
 ...
 [ 629  294  176]
 [ 167   96  651]
 [ 160  185  714]]


## 4. 生成批测试集三元组
下一步是产生测试集三元组[u, i, j]。对于每个用户u，它的评分电影 i 是我们在user_ratings_test中随机抽取的，它的 j 是用户 u 所有没有评分过的电影集合，比如用户 u 有1000部电影没有评分，那么这里该用户的测试集样本就有1000个。

In [6]:
# 定义函数，生成测试集
def generate_test_batch(user_ratings, user_ratings_test, n_item):
    
    # 遍历每个用户
    for u in user_ratings.keys():  
        test = []
        i = user_ratings_test[u]  # 随机抽取的一个已评分item
        for j in range(1, n_item+1):
            if not (j in user_ratings[u]):  # 获得该用户所有未评分过的item
                test.append([u, i, j])
        yield np.asarray(test)
        
# 调用函数
for uij in generate_test_batch(user_ratings, user_ratings_test, n_item):
    print(uij)

[[ 196 1118    1]
 [ 196 1118    2]
 [ 196 1118    3]
 ...
 [ 196 1118 1680]
 [ 196 1118 1681]
 [ 196 1118 1682]]
[[ 186  257    1]
 [ 186  257    2]
 [ 186  257    3]
 ...
 [ 186  257 1680]
 [ 186  257 1681]
 [ 186  257 1682]]
[[  22  430    1]
 [  22  430    3]
 [  22  430    5]
 ...
 [  22  430 1680]
 [  22  430 1681]
 [  22  430 1682]]
[[ 244  324    2]
 [ 244  324    4]
 [ 244  324    5]
 ...
 [ 244  324 1680]
 [ 244  324 1681]
 [ 244  324 1682]]
[[ 166  984    1]
 [ 166  984    2]
 [ 166  984    3]
 ...
 [ 166  984 1680]
 [ 166  984 1681]
 [ 166  984 1682]]
[[ 298  527    2]
 [ 298  527    3]
 [ 298  527    4]
 ...
 [ 298  527 1680]
 [ 298  527 1681]
 [ 298  527 1682]]
[[ 115  922    1]
 [ 115  922    2]
 [ 115  922    3]
 ...
 [ 115  922 1680]
 [ 115  922 1681]
 [ 115  922 1682]]
[[ 253  188    2]
 [ 253  188    3]
 [ 253  188    5]
 ...
 [ 253  188 1680]
 [ 253  188 1681]
 [ 253  188 1682]]
[[ 305  131    3]
 [ 305  131    4]
 [ 305  131    5]
 ...
 [ 305  131 1680]
 [ 305  131

[[  91  343    1]
 [  91  343    2]
 [  91  343    3]
 ...
 [  91  343 1680]
 [  91  343 1681]
 [  91  343 1682]]
[[ 294  752    2]
 [ 294  752    3]
 [ 294  752    4]
 ...
 [ 294  752 1680]
 [ 294  752 1681]
 [ 294  752 1682]]
[[ 229  269    1]
 [ 229  269    2]
 [ 229  269    3]
 ...
 [ 229  269 1680]
 [ 229  269 1681]
 [ 229  269 1682]]
[[  36  748    1]
 [  36  748    2]
 [  36  748    3]
 ...
 [  36  748 1680]
 [  36  748 1681]
 [  36  748 1682]]
[[  70  546    2]
 [  70  546    3]
 [  70  546    4]
 ...
 [  70  546 1680]
 [  70  546 1681]
 [  70  546 1682]]
[[  14   56    1]
 [  14   56    2]
 [  14   56    3]
 ...
 [  14   56 1680]
 [  14   56 1681]
 [  14   56 1682]]
[[ 295   67    2]
 [ 295   67    3]
 [ 295   67    5]
 ...
 [ 295   67 1680]
 [ 295   67 1681]
 [ 295   67 1682]]
[[ 233  515    1]
 [ 233  515    2]
 [ 233  515    3]
 ...
 [ 233  515 1680]
 [ 233  515 1681]
 [ 233  515 1682]]
[[ 214   92    1]
 [ 214   92    2]
 [ 214   92    3]
 ...
 [ 214   92 1680]
 [ 214   92

## 5. 构建BPR数据流
有了训练集和测试集，下面是用TensorFlow构建BPR算法的数据流，其中hidden_dim就是矩阵分解的隐含维度k。

In [7]:
def bpr_mf(user_count, item_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        user_emb_w = tf.get_variable("user_emb_w", [user_count+1, hidden_dim], initializer=tf.random_normal_initializer(0, 0.1))
        item_emb_w = tf.get_variable("item_emb_w", [item_count+1, hidden_dim], initializer=tf.random_normal_initializer(0, 0.1))
        
        u_emb = tf.nn.embedding_lookup(user_emb_w, u)
        i_emb = tf.nn.embedding_lookup(item_emb_w, i)
        j_emb = tf.nn.embedding_lookup(item_emb_w, j)
    
    x = tf.reduce_sum(tf.multiply(u_emb, (i_emb - j_emb)), 1, keep_dims=True)
    
    mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    
    l2_norm = tf.add_n([
            tf.reduce_sum(tf.multiply(u_emb, u_emb)), 
            tf.reduce_sum(tf.multiply(i_emb, i_emb)),
            tf.reduce_sum(tf.multiply(j_emb, j_emb))
        ])
    
    regulation_rate = 0.0001
    bprloss = regulation_rate * l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(x)))
    
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(bprloss)
    return u, i, j, mf_auc, bprloss, train_op

## 6. 训练与测试

In [9]:
user_count = n_user
item_count = n_item

with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, mf_auc, bprloss, train_op = bpr_mf(user_count, item_count, 20)
    session.run(tf.initialize_all_variables())
    for epoch in range(1, 10):
        _batch_bprloss = 0
        for k in range(1, 1000): # uniform samples from training set
            uij = generate_train_batch(user_ratings, user_ratings_test, item_count)

            _bprloss, _train_op = session.run([bprloss, train_op], feed_dict={u:uij[:,0], i:uij[:,1], j:uij[:,2]})
            _batch_bprloss += _bprloss
        
        print ("epoch: ", epoch)
        print ("bpr_loss: ", _batch_bprloss / k)
        print ("_train_op")
        
        
        user_count = 0
        _auc_sum = 0.0

        # each batch will return only one user's auc
        for t_uij in generate_test_batch(user_ratings, user_ratings_test, item_count):

            _auc, _test_bprloss = session.run([mf_auc, bprloss], feed_dict={u:t_uij[:,0], i:t_uij[:,1], j:t_uij[:,2]})
            user_count += 1
            _auc_sum += _auc
        print ("test_loss: ", _test_bprloss, "test_auc: ", _auc_sum/user_count)
        print ("")
        
        
    variable_names = [v.name for v in tf.trainable_variables()]
    values = session.run(variable_names)
    for k,v in zip(variable_names, values):
        print("Variable: ", k)
        print("Shape: ", v.shape)
        print(v)

epoch:  1
bpr_loss:  0.7245672167361797
_train_op
test_loss:  0.7961966 test_auc:  0.503310084850715

epoch:  2
bpr_loss:  0.7243792564422638
_train_op
test_loss:  0.79589766 test_auc:  0.5032485600340912

epoch:  3
bpr_loss:  0.7242160638770064
_train_op
test_loss:  0.7956418 test_auc:  0.5032309624698549

epoch:  4
bpr_loss:  0.7241332659850249
_train_op
test_loss:  0.7953759 test_auc:  0.5032066674756127

epoch:  5
bpr_loss:  0.7239358831454326
_train_op
test_loss:  0.7951246 test_auc:  0.5031469301923676

epoch:  6
bpr_loss:  0.7238392424774361
_train_op
test_loss:  0.79488367 test_auc:  0.5031179064199129

epoch:  7
bpr_loss:  0.7237565287121304
_train_op
test_loss:  0.7946097 test_auc:  0.5030772203389676

epoch:  8
bpr_loss:  0.7236536236496659
_train_op
test_loss:  0.794351 test_auc:  0.5030377772536715

epoch:  9
bpr_loss:  0.7234494624075828
_train_op
test_loss:  0.794073 test_auc:  0.5029830295108121

Variable:  user_emb_w:0
Shape:  (944, 20)
[[-0.13892458 -0.18152796  0.056